***Readme***

This code helps calculate the current value for a meta goal based on how far above the road are latest datapoints across a list of goals.

 Before using, mark the list of the goals which you need to take into account with a unique tag. (It only makes sense to take into account some goals and not others. For example, it doesn't work for do-less goals. Works best for goals where the direction of data accumulation is monotonous. E.g. odometer or do-more goals where the total can only become higher and whittle-down goals where the total can only become lower. Should work fine with odometer resets.)

Set up a goal that will accept the meta values as an odometer goal. I use the rate of 0.1, just for it to be non-zero, but the rate is not important here, since the pressure will be not from this goal's rate but from the rates of all the goals that are taken into account.

The datapoint that is posted takes into account 10 goals with closest-to-the-road latest datapoints. Think of it as a score on the scale of 0 to 100, where 0 is "at least 10 goals are in the red today" and 100 as "all goals are at least 10 days ahead of the plan". In my experience, it make little sense to actually strive to achieve the 90+ range, as then the freedom to work more one this or that goal disappears. Instead, it is the journey that is the destination :)

**Add your INFO in the second cell with code.**

After the last cell, you cap see a recap confirming that the datapoint has been posted and a list of goals that you can work on that contribute to the meta goal.

ChatGPT's recap of what it helped me do:

We developed a code pipeline in Python to interact with Beeminder’s API and analyze goal data, starting by defining key parameters such as USERNAME, ACCESS_TOKEN, and BASE_URL. Using this setup, we retrieved all goals data from Beeminder and created a DataFrame to display key goal attributes. We then filtered the goals based on a specific tag ("normal") and used this filtered data to extract future goal rates and dynamically added a "rates_future" column.

Next, we calculated "max_r_future", which either took the highest or lowest value from "rates_future" based on the sign of its entries, with manual input for mixed values. We then adjusted this rate based on goal units ("d", "w", "m", "y") to create a normalized "max_r_future_adj" column. We also calculated "ahead_by_d" to show how far ahead a goal is, and rounded this value down to create an integer column, "ahead_by_d_int". Finally, we summarized the data in two variables, qu_below_10 and sum_10_below_10, reflecting how many goals were close to derailing and the adjusted sum of the lowest values.

For display, we created a new DataFrame truncated_df, formatted columns to show up to two decimals, and posted the summary data to Beeminder’s API. We wrapped up by displaying selected columns in a human-readable format, creating a well-rounded tool for monitoring Beeminder goals.

In [1]:
import requests

In [2]:
# Add your info here!!!!

USERNAME = 'username'
ACCESS_TOKEN = 'XXXXXXXXX'
TAG_FOR_LIST = 'xxxxxxxxx'
METAGOAL_SLUG = 'xxxxxxxxx'

In [4]:
# Base URL for Beeminder API
BASE_URL = 'https://www.beeminder.com/api/v1/'

In [ ]:
# Define the endpoint URL to retrieve all goals
goals_url = f"{BASE_URL}/users/{USERNAME}/goals.json"

# Function to make a GET request and retrieve all goals
def get_all_goals():
    response = requests.get(goals_url, params={"auth_token": ACCESS_TOKEN})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve goals: {response.status_code}")
        return None

# Test the function
all_goals = get_all_goals()
all_goals  # Display the retrieved goals


In [ ]:
import pandas as pd

# Function to convert the retrieved goals data to a DataFrame with error checking
def goals_to_dataframe(goals_data):
    # Define the columns we want to extract
    desired_columns = ['slug', 'losedate', 'fullroad', 'delta', 'runits', 'tags', 'goal_type']
    
    # Check if goals_data is not None
    if goals_data:
        # Identify missing columns
        missing_columns = [col for col in desired_columns if col not in goals_data[0]]
        if missing_columns:
            print(f"Warning: The following columns are missing from the data and will not be included: {missing_columns}")
        
        # Extract only the available specified columns
        available_columns = [col for col in desired_columns if col in goals_data[0]]
        goals_df = pd.DataFrame(goals_data)[available_columns]
        return goals_df
    else:
        print("No data available to convert to DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame if there's no data

# Populate the DataFrame with the goals data
goals_df = goals_to_dataframe(all_goals)
goals_df  # Display the DataFrame


In [ ]:
# Filter rows where 'tags' contains TAG_FOR_LIST and 'goal_type' is not 'drinker'
normal_goals_df = goals_df[
    goals_df['tags'].apply(lambda tags: TAG_FOR_LIST in tags if tags else False) & 
    (goals_df['goal_type'] != 'drinker')
]

# Display the filtered DataFrame
normal_goals_df


In [ ]:
import time

# Get the current time as a Unix timestamp
current_timestamp = int(time.time())

# Display the current timestamp
current_timestamp

In [ ]:
# Function to extract future rates based on current timestamp
def get_future_rates(fullroad, current_timestamp):
    # Filter for rates where the timestamp is in the future
    return [entry[-1] for entry in fullroad if entry[0] > current_timestamp]

# Copy DataFrame to avoid SettingWithCopyWarning and apply future rates extraction
normal_goals_df = normal_goals_df.copy()  # Make an explicit copy
normal_goals_df['rates_future'] = normal_goals_df['fullroad'].apply(lambda fullroad: get_future_rates(fullroad, current_timestamp))

# Display the updated DataFrame with the new 'rates_future' column
normal_goals_df


In [ ]:
# Function to determine max_r_future based on specified conditions
def determine_max_r_future(slug, rates_future):
    if all(rate >= 0 for rate in rates_future):
        return max(rates_future)  # All non-negative, take max
    elif all(rate <= 0 for rate in rates_future):
        return min(rates_future)  # All non-positive, take min
    else:
        # Mixed values, prompt the user for input
        print(f"The goal '{slug}' has mixed rates in 'rates_future': {rates_future}")
        while True:
            try:
                user_input = float(input(f"Please enter a value to use for '{slug}': "))
                return user_input
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

# Apply the function to each row in normal_goals_df and add as a new column
normal_goals_df['max_r_future'] = normal_goals_df.apply(
    lambda row: determine_max_r_future(row['slug'], row['rates_future']), axis=1
)

# Display the updated DataFrame
normal_goals_df


In [ ]:
# Filter out rows where 'max_r_future' is 0
normal_goals_df = normal_goals_df.loc[normal_goals_df['max_r_future'] != 0]

# Display the updated DataFrame
normal_goals_df


In [ ]:
# Function to adjust max_r_future based on runits
def adjust_max_r_future(max_r_future, runits):
    if runits == "d":
        return max_r_future
    elif runits == "w":
        return max_r_future / 7
    elif runits == "m":
        return max_r_future / 30
    elif runits == "y":
        return max_r_future / 365
    else:
        return max_r_future  # Default in case of unexpected 'runits' value

# Apply the adjustment function to create the 'max_r_future_adj' column
normal_goals_df['max_r_future_adj'] = normal_goals_df.apply(
    lambda row: adjust_max_r_future(row['max_r_future'], row['runits']), axis=1
)

# Display the updated DataFrame
normal_goals_df


In [ ]:
# Function to calculate 'ahead_by_d' based on delta and max_r_future_adj
def calculate_ahead_by_d(delta, max_r_future_adj):
    # Check if delta and max_r_future_adj have opposite signs
    if (delta < 0 < max_r_future_adj) or (delta > 0 > max_r_future_adj):
        return 0  # Opposite signs, set to 0
    else:
        return delta / max_r_future_adj + 1  # Same sign, calculate the value

# Apply the function to each row to create the 'ahead_by_d' column
normal_goals_df['ahead_by_d'] = normal_goals_df.apply(
    lambda row: calculate_ahead_by_d(row['delta'], row['max_r_future_adj']), axis=1
)

# Display the updated DataFrame
normal_goals_df


In [ ]:
import math

# Create 'ahead_by_d_int' by rounding down 'ahead_by_d' to the nearest integer
normal_goals_df['ahead_by_d_int'] = normal_goals_df['ahead_by_d'].apply(math.floor)

# Display the updated DataFrame
normal_goals_df


In [ ]:
# Count the number of rows where 'ahead_by_d_int' is less than 10
qu_below_10 = (normal_goals_df['ahead_by_d_int'] < 10).sum()

# Get the 10 lowest values in 'ahead_by_d_int' and apply the min condition for summing
sum_10_below_10 = normal_goals_df['ahead_by_d_int'].nsmallest(10).apply(lambda x: min(x, 10)).sum()

# Display the results
qu_below_10, sum_10_below_10


In [ ]:
import requests
from datetime import datetime

# Function to post a datapoint to Beeminder
def post_to_beeminder(goal_slug, value, comment):
    url = f"{BASE_URL}/users/{USERNAME}/goals/{goal_slug}/datapoints.json"
    data = {
        "auth_token": ACCESS_TOKEN,
        "value": value,
        "comment": comment
    }
    
    # Make the POST request
    response = requests.post(url, data=data)
    
    # Check if the request was successful
    if response.status_code == 200:
        print("Datapoint added successfully!")
    else:
        print(f"Failed to add datapoint: {response.status_code} - {response.json()}")

# Define the goal slug and prepare the values for posting
goal_slug = METAGOAL_SLUG
value = sum_10_below_10
current_time = datetime.now().strftime("%Y-%m-%d %H:%M")
comment = f"{qu_below_10} goals <10. Upd at {current_time}"

# Post to Beeminder
post_to_beeminder(goal_slug, value, comment)


In [ ]:
from datetime import timedelta

# Function to calculate "derail_in" as "in D days HH:MM"
def calculate_derail_in(losedate, current_timestamp):
    # Calculate the time difference between losedate and the current time
    time_difference = losedate - current_timestamp
    if time_difference <= 0:
        return "Already derailed"  # Handle past or current losedate
    # Convert time difference to days, hours, and minutes
    delta = timedelta(seconds=time_difference)
    days = delta.days
    hours, remainder = divmod(delta.seconds, 3600)
    minutes = remainder // 60
    return f"in {days} days {hours:02}:{minutes:02}"

# Create a truncated copy of the DataFrame
truncated_df = normal_goals_df.drop(columns=['fullroad', 'runits', 'tags', 'rates_future', 'max_r_future', 'goal_type']).copy()

# Calculate "derail_in" based on "losedate" and the current timestamp, then add it to the DataFrame
truncated_df['derail_in'] = truncated_df['losedate'].apply(lambda losedate: calculate_derail_in(losedate, current_timestamp))

# Drop the "losedate" column as requested
truncated_df = truncated_df.drop(columns=['losedate'])

# Display the truncated DataFrame
truncated_df


In [ ]:
# Function to format floats to 2 decimal places or as integers if they are close enough
def format_float(value):
    # Check if the value is already a string to avoid re-formatting
    if isinstance(value, str):
        return value
    
    # Check if the value is within 0.01 of its nearest integer
    if abs(value - round(value)) < 0.01:
        return int(round(value))  # Display as an integer if close enough
    else:
        return f"{value:.2f}"  # Otherwise, display with 2 decimal places

# Apply the formatting function to each specified column
truncated_df['delta'] = truncated_df['delta'].apply(format_float)
truncated_df['max_r_future_adj'] = truncated_df['max_r_future_adj'].apply(format_float)
truncated_df['ahead_by_d'] = truncated_df['ahead_by_d'].apply(format_float)

# Display the nicely formatted DataFrame
truncated_df


In [ ]:
import time

# Wait for 4 seconds
time.sleep(4)

# Function to get specific attributes from a Beeminder goal
def get_goal_attributes(goal_slug, attributes):
    url = f"{BASE_URL}/users/{USERNAME}/goals/{goal_slug}.json"
    response = requests.get(url, params={"auth_token": ACCESS_TOKEN})
    
    if response.status_code == 200:
        goal_data = response.json()
        # Extract the specified attributes
        return {attr: goal_data.get(attr) for attr in attributes}
    else:
        print(f"Failed to retrieve goal data: {response.status_code}")
        return None

# Retrieve "curval" and "limsum" for the specified goal
goal_slug = METAGOAL_SLUG
attributes = ["curval", "limsum"]
goal_info = get_goal_attributes(goal_slug, attributes)

# Print the results
if goal_info:
    print("curval:", goal_info["curval"])
    print("limsum:", goal_info["limsum"])


truncated_df
